# Analisis Exploratorio

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv('vehiculos_train.csv')

In [8]:
df.head()

,marca,modelo,color,anio_registro,tipo_vehiculo,millas_recorridas,tamanio_motor,transmision,tipo_combustible,precio_vehiculo,num_asientos,num_puertas,problema_averia,id_problema_averia,fecha_averia,complejidad_reparacion,costo_reparacion,horas_reparacion,fecha_reparacion,fraude
0,Jeep,Renegade,Black,2018.0,SUV,17.0,1.4L,Manual,Petrol,16000.0,5.0,5.0,Electrical Issue,5,2018-06-07,1,76.0000,3.0,2018-06-07,0
1,Vauxhall,Insignia,Grey,2011.0,Hatchback,93000.0,NaN,Manual,Petrol,3995.0,5.0,5.0,Excessive Emissions,0,2018-08-09,1,23.5955,1.0,2018-08-09,0
2,Mercedes-Benz,GLC Class,White,2019.0,SUV,6879.0,2.0L,Automatic,Diesel,31498.0,5.0,5.0,Windscreen Crack,0,2021-06-17,1,38.8988,1.0,2021-06-17,0
3,Renault,NaN,NaN,2014.0,Hatchback,20669.0,1.2L,Manual,Petrol,6950.0,5.0,5.0,Brake Pads Worn,0,2018-04-23,1,43.4750,2.0,2018-04-23,0
4,Ford,C-Max,NaN,2007.0,MPV,84786.0,1.6L,Manual,Petrol,NaN,5.0,5.0,Tyre Alignment,0,2018-05-20,1,10.2600,0.5,2018-05-20,0


# Tarea 3

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [34]:
df_temp = df.drop(columns=['id_problema_averia'])
X_train, X_val, y_train, y_val = train_test_split(df_temp.drop(columns='fraude'), df_temp.fraude, test_size=0.2, random_state=0)

In [35]:
categoricalColumns = ['marca', 'color', 'modelo', 'tipo_vehiculo', 'transmision',
                      'tipo_combustible', 'problema_averia',
                      'tamanio_motor', 'complejidad_reparacion']
negColumns = ['num_puertas', 'anio_registro', 'millas_recorridas', 'precio_vehiculo', 'costo_reparacion', 'horas_reparacion']

#Imputa valores basado en la moda basada en Marca, tipo vehiculo y modelo.
categoricalImputer = Pipeline([
    ('oneHot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

non_linear_transformers = ColumnTransformer(
    [
        ('log_transform', FunctionTransformer(lambda x: np.log1p(x), feature_names_out='one-to-one'), ['precio_vehiculo', 'costo_reparacion', 'horas_reparacion']),
        ('sqrt_transform', FunctionTransformer(lambda x: np.sqrt(x), feature_names_out='one-to-one'), ['millas_recorridas'])
    ],
    remainder='passthrough' # Mantiene cualquier columna que no esté en log_cols o sqrt_cols
)

negativeValues = Pipeline([
    ('non_linear_transform', non_linear_transformers)
])

preProcess_model1 = ColumnTransformer([
    ('cat', categoricalImputer, categoricalColumns),
    ('neg', negativeValues, negColumns + ["marca", "modelo", "tipo_vehiculo"])
], remainder='drop')

In [36]:
asd = preProcess_model1.fit_transform(X_train, y_train)

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in sqrt
  result = func(self.values, **kwargs)
